# Parameter optimization example

### Preamble

In this example we will use the `LikelihoodVectorized` class to make a simple but effective wrapper for function and gradient calls for scipy optimizer methods.

### Initialize Ray

We use the `ray` package a lot. So we start off by initializing an instance of ray. Note the `num_cpus` argument, which should be adapted for your machine.

In [1]:
import ray
ray.init(
    num_cpus=32,
    #_temp_dir="/scratch/wulsdorf/scratch"
)

2023-05-22 12:07:11,938	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Python version:,3.9.10
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8266


### Load the dataset

Next, we load the dataset. This is a set of alkane molecules, kindly provided by Trevor Gokey.

By default, the initial MM parameters assigned to the molecules in the systems are taken from the OpenFF 1.3.0 Parsley forcefield. If you want to change to a different forcefield, set forcefield_name to a different value.

In [116]:
import pickle
from BayesTyper import system
from BayesTyper import tools

with open("./AlkaneSet/opt-training-1.pickle", "rb") as fopen:
    optdataset_dict = pickle.load(fopen)

systemmanager = system.SystemManager()
tools.generate_systemmanager(
    list(
        optdataset_dict.keys()
    ),
    systemmanager,
    optdataset_dict=optdataset_dict,
    torsiondataset_dict=None,
    weight_geo = 1.,
    weight_vib = 0,
    weight_offeq = 0,
    weight_torsion = 0,
    weight_force = 1.,
    add_units=True,
    ene_weighting=False,
    force_projection=False,
)

Adding C[C@H]1CCCCCCC(C)(C)C1
Adding C[C@@H]1CCC[C@@H](C)C1
Adding CCCCCC(C)(C)C
Adding C[C@H]1CCC[C@@H](C)C1
Adding CCC(C)(C)CC(C)C
Adding C[C@@H]1CC[C@H]1C
Adding CC(C)CC(C)(C)C
Adding C[C@H]1CCCCC[C@H]1C
Adding CC(C)CCCC(C)(C)C
Adding C[C@@H]1CCCCC1(C)C
Adding CC1CCC1
Adding CC(C)CCC(C)(C)C
Adding CCC(C)(C)C(C)(C)C
Adding CCCC(C)(C)C(C)(C)C
Adding C[C@H]1CCCCC[C@@H]1C
Adding C[C@H]1CC[C@H](C)C1
Adding CC(C)[C@@H](C)C(C)(C)C
Adding CC(C)CCC(C)C
Adding CC1(C)CCCCC1(C)C
Adding CCCC(C)CCC
Adding C[C@H]1CCCCCC[C@H]1C
Adding CCCC(C)(C)CC(C)C
Adding CC1(C)CCCC(C)(C)C1
Adding CC1(C)CCCCCCCC1
Adding CC(C)C(C)(C)C
Adding C1CC1
Adding CC[C@H](C)CC(C)(C)C
Adding CC[C@H](C)CC(C)C
Adding CC1(C)CCCCCC(C)(C)C1
Adding C[C@H]1CCCCC(C)(C)C1
Adding CCCC(C)C
Adding C[C@H]1CCCC1(C)C
Adding CC1(C)CCCCCC1(C)C
Adding CC1(C)CCCCCCC(C)(C)C1
Adding CC(C)[C@@H](C)CC(C)(C)C
Adding CCC(C)(C)CC(C)(C)C
Adding CCCCCC(C)C
Adding C1CCC1
Adding C[C@H]1CCCCCC1(C)C
Adding C[C@H]1CC[C@@H](C)C1
Adding CCCC(C)(C)C(C)C
Addin

(<BayesTyper.system.SystemManager at 0x7efd9b120970>, 14353.0)

### Generate bond and angle vectors

Next, we will generate useful high-level parameter vector objects that we can use to manipulate the parameters in the systems objects. We will create one parameter vector for bond forces and one for angle forces. Note that the parameter vectors expose only a linear mapping to the actual parameter vectors (very much in the sense that ForceBalance treats parameter values).

In [117]:
from BayesTyper import parameters
from BayesTyper import vectors

bond_mngr = parameters.BondManager()
bond_mngr.add_system_manager(systemmanager)

angle_mngr = parameters.AngleManager()
angle_mngr.add_system_manager(systemmanager)

pvec_bonds  = vectors.ForceFieldParameterVector(
    bond_mngr,
    parameter_name_list=["k", "length"],
    scaling_list=[1.e5, 1.e-1]
)
pvec_angles = vectors.ForceFieldParameterVector(
    angle_mngr,
    parameter_name_list=["k", "angle"],
    scaling_list=[1.e2, 1.e1]
)

### Define an objective function and a gradient

Next, we want to use `LikelihoodVectorized` to generate a gradient and objective function. We will also include a prior function, which is easy to implement due to the linear mapping of the parameter vector.

In [147]:
from BayesTyper import likelihoods
from BayesTyper import targets
import numpy as np

logL = likelihoods.LikelihoodVectorized(
    [pvec_bonds, pvec_angles],
    targets.TargetComputer(systemmanager),
    N_sys_per_batch=4,
    three_point=False,
)

### This number just scales the prior
s = 1.

def grad(x):

    dl = logL.grad(
        x, 
        grad_diff=1.e-4, 
        use_jac=False
    )
    dp = 2. * s * x
    dy = -dl - dp
    return dy

def func(x):

    l = logL(x)
    ### This is the prior
    p = s * np.sum(x**2)
    y = -l - p
    return y

### Optimize

Next, we will use a standard scipy optimizer to minimize the objective function. Note that we are already starting from a pretty good forcefield. So might not get far here.

In [148]:
from scipy import optimize

x0  = logL.pvec
x0[:] = 0.
print(
    "Initial f:", func(x0)
)
        
result = optimize.minimize(
    x0=x0,
    fun=func,
    jac=grad,
    method="BFGS",
    options = {
        'gtol': 1e-6, 
        'disp': True
        },
)

Initial f: 30220.16396130976
         Current function value: 28448.868204
         Iterations: 2
         Function evaluations: 35
         Gradient evaluations: 26


In [149]:
### Runtime 6 minutes
### Runtime w/o logL grads, 30 minutes
result

      fun: 28448.868203534323
 hess_inv: array([[ 9.99931949e-01,  2.70660291e-04, -7.09995823e-05,
        -1.31368480e-03, -4.07851106e-03, -4.07278266e-03,
        -3.74248924e-03, -3.71708696e-03,  6.79058375e-04,
         6.81055276e-04,  1.02272942e-03,  1.02471883e-03,
         1.02449237e-03,  1.02352093e-03, -1.66436071e-04,
        -1.66480083e-04, -1.66266599e-04, -1.66430559e-04,
        -1.66272517e-04, -1.66458603e-04, -2.05385991e-04,
        -2.05307277e-04],
       [ 2.70660291e-04,  9.86384844e-01,  4.89392327e-04,
         5.07781414e-02,  5.12914548e-03,  4.84814036e-03,
        -9.03161568e-03, -1.05936706e-02, -3.22225476e-02,
        -3.23403667e-02, -4.68689156e-02, -4.69827750e-02,
        -4.69727223e-02, -4.69040269e-02,  2.44705433e-03,
         2.44969770e-03,  2.43645497e-03,  2.44536912e-03,
         2.43649965e-03,  2.44845052e-03,  5.60115085e-03,
         5.59681016e-03],
       [-7.09995823e-05,  4.89392327e-04,  9.99928405e-01,
        -1.77508140e-0

### Check parameter vectors after the fitting

After fitting, the relaxed parameter vectors for bonds and angles have the following values

In [154]:
logL.apply_changes(result.x)
pvec_bonds.vector_k

array([Quantity(value=216408.67037852504, unit=kilojoule/(nanometer**2*mole)),
       Quantity(value=0.1519438642243602, unit=nanometer),
       Quantity(value=315507.09968668834, unit=kilojoule/(nanometer**2*mole)),
       Quantity(value=0.10978302284678046, unit=nanometer)], dtype=object)

In [155]:
pvec_angles.vector_k

array([Quantity(value=415.3582246186403, unit=kilojoule/(mole*radian**2)),
       Quantity(value=113.67315972716847, unit=degree),
       Quantity(value=278.4016765421681, unit=kilojoule/(mole*radian**2)),
       Quantity(value=114.28903972300554, unit=degree),
       Quantity(value=675.9661505932731, unit=kilojoule/(mole*radian**2)),
       Quantity(value=116.57771830055098, unit=degree),
       Quantity(value=416.14461225889454, unit=kilojoule/(mole*radian**2)),
       Quantity(value=115.79475480619591, unit=degree),
       Quantity(value=292.0523490496428, unit=kilojoule/(mole*radian**2)),
       Quantity(value=113.68507214134416, unit=degree),
       Quantity(value=1030.7802739599806, unit=kilojoule/(mole*radian**2)),
       Quantity(value=68.77999029352144, unit=degree),
       Quantity(value=271.2357637195792, unit=kilojoule/(mole*radian**2)),
       Quantity(value=119.17074470254688, unit=degree),
       Quantity(value=121.42007094636824, unit=kilojoule/(mole*radian**2)),
      